# Loading Packages

In [ ]:
!pip install transformers

In [ ]:
!pip install sentence-transformers

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf

import nltk
import re

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

--2022-02-02 11:05:59--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.138.21
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.138.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  16.7MB/s    in 2m 2s   

2022-02-02 11:08:03 (12.8 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
!gunzip GoogleNews-vectors-negative300.bin

gzip: GoogleNews-vectors-negative300.bin: unknown suffix -- ignored


In [ ]:
!ls -l --block-size=MB

total 5292MB
-rw-r--r-- 1 root root 3645MB Mar  5  2015 GoogleNews-vectors-negative300.bin
-rw-r--r-- 1 root root 1648MB Mar  5  2015 GoogleNews-vectors-negative300.bin.gz
-rw-r--r-- 1 root root    1MB Feb  2 10:59 linkedin.csv
drwxr-xr-x 1 root root    1MB Jan  7 14:33 sample_data
-rw-r--r-- 1 root root    1MB Feb  2 10:59 twitter.csv


# Data Loading

In [ ]:
df_twitter = pd.read_csv('/content/twitter.csv')
df_linkedin = pd.read_csv('/content/linkedin.csv')

In [ ]:
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268 entries, 0 to 267
Data columns (total 23 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Unnamed: 0                                   268 non-null    int64  
 1   data_id                                      268 non-null    int64  
 2   data_text                                    268 non-null    object 
 3   data_non_public_metrics_impression_count     268 non-null    int64  
 4   data_non_public_metrics_url_link_clicks      210 non-null    float64
 5   data_non_public_metrics_user_profile_clicks  268 non-null    int64  
 6   data_organic_metrics_impression_count        268 non-null    int64  
 7   data_organic_metrics_like_count              268 non-null    int64  
 8   data_organic_metrics_reply_count             268 non-null    int64  
 9   data_organic_metrics_retweet_count           268 non-null    int64  
 10  da

In [ ]:
df_twitter = df_twitter.rename(columns={'data_id':'idt', 'norm_twitter_posts':'processed_text'})
df_twitter = df_twitter.drop_duplicates(subset="idt")

In [ ]:
df_linkedin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   273 non-null    int64  
 1   DateTime                     273 non-null    object 
 2   LinkeInPage                  273 non-null    object 
 3   DataID                       273 non-null    int64  
 4   Tags                         97 non-null     object 
 5   PostCampaign                 0 non-null      float64
 6   PostType                     273 non-null    object 
 7   PostMessage                  273 non-null    object 
 8   Reactions                    273 non-null    int64  
 9   Impressions                  273 non-null    int64  
 10  Engagement                   273 non-null    float64
 11  Clicks                       273 non-null    int64  
 12  Shares                       273 non-null    int64  
 13  Comments            

In [ ]:
df_linkedin = df_linkedin.rename(columns={'DataID':'idl', 'norm_linkedin_posts':'processed_text'})
df_linkedin = df_linkedin.drop_duplicates(subset="idl")

In [ ]:
df_t = df_twitter[['idt','processed_text']]
df_t["idl"] = 0

In [ ]:
df_l = df_linkedin[['idl','processed_text']]
df_l["idt"] = 0

In [ ]:
frames = [df_t, df_l] 
df = pd.concat(frames)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 541 entries, 0 to 272
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   idt             541 non-null    int64 
 1   processed_text  536 non-null    object
 2   idl             541 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 16.9+ KB


In [ ]:
df.isna().sum()

idt               0
processed_text    5
idl               0
dtype: int64

In [ ]:
df.dropna(subset=['processed_text'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536 entries, 0 to 272
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   idt             536 non-null    int64 
 1   processed_text  536 non-null    object
 2   idl             536 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 16.8+ KB


In [ ]:
data = df.processed_text.values.tolist()

In [ ]:
from wordcloud import WordCloud
all_words = ''.join([word for word in data])
all_words
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)
plt.figure(figsize=(15, 8))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title("Some frequent words used in the posts", weight='bold', fontsize=14)
plt.show()

In [ ]:
def get_top_n_words(corpus, n=10):
  vec = CountVectorizer(stop_words='english').fit(corpus)
  bag_of_words = vec.transform(corpus)
  sum_words = bag_of_words.sum(axis=0) 
  words_freq = [(word, sum_words[0, idx]) for word, idx in   vec.vocabulary_.items()]
  words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
  return words_freq[:n]
words = []
word_values = []
for i,j in get_top_n_words(data,15):
  words.append(i)
  word_values.append(j)
fig, ax = plt.subplots(figsize=(16,8))
ax.bar(range(len(words)), word_values);
ax.set_xticks(range(len(words)));
ax.set_xticklabels(words, rotation='vertical');
ax.set_title('Top 15 words in the posts dataset');
ax.set_xlabel('Word');
ax.set_ylabel('Number of occurences');
plt.show()

In [ ]:
import nltk

tokenized_corpus = [nltk.word_tokenize(doc) for doc in data]

# Load Word2Vec Embeddings in a Word2Vec Model

In [ ]:
from gensim.models import KeyedVectors

w2v_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
def average_word_vectors(words, model, vocabulary, num_features):
    
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector


def averaged_word_vectorizer(corpus, model, num_features):
    vocabulary = set(model.index2word)
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [ ]:
embeddings = averaged_word_vectorizer(tokenized_corpus, model=w2v_model, num_features=300)

In [ ]:
embeddings.shape

(536, 300)

# Clustering

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans

km = KMeans(
    n_clusters=4, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=0
)
y_km = km.fit_predict(embeddings)
dff = pd.DataFrame({'posts' :df['processed_text'], 'idt' :df['idt'], 'idl' :df['idl'], 'topic_cluster' :y_km })

# c-TF-IDF

Class-based TF-IDF score

In [ ]:
docs_df = df.rename(columns={'processed_text':"Doc"})
docs_df['Topic'] = y_km

In [ ]:
docs_df['Topic'].value_counts()

3    242
0    132
1     93
2     69
Name: Topic, dtype: int64

First create a single document for each cluster of documents:

In [ ]:
docs_per_topic = docs_df.groupby(['Topic'])['Doc'].apply(list).reset_index()

In [ ]:
docs_per_topic['Doc'] = docs_per_topic['Doc'].apply(lambda x: ' '.join(map(str, x)))

Apply the class-based TF-IDF:

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count
  
tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m=len(data))

# Topic Representation

In [ ]:
def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .Doc
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "Doc": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)
topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Topic,Size
3,3,242
0,0,132
1,1,93
2,2,69


In [ ]:
# Visualizing wrods belonging to each topic
top_n_words